In [110]:
# Import des librairies
import requests
import re

import openmeteo_requests

import pandas as pd
import numpy as np
import requests_cache
from retry_requests import retry
import time


In [111]:
df = pd.read_csv('BDD_PC_PotentielSolaire_Annuel.csv')
df.head()

,Unnamed: 0,Nom,Lien,Début,Fin,Distance,Type voie,Nature voie,Revêtement,Carte,Coordonnées de début,Coordonnées de fin,Coordonnées milieu,Énergie produite annuelle (kWh / m2 de panneau)
0,0,Voie Verte du Chemin d’Encochebérot,https://www.af3v.org/les-voies-vertes/voies/12...,L ISLE JOURDAIN,L ISLE JOURDAIN,"1,6km",Voie verte lisse,voie verte,Enrobé,https://sig.af3v.org/index.php/view/embed/?rep...,"('1.09673702748082', '43.62109781579031')","('1.1131950619939002', '43.6267673096849')","(1.1049660447373602, 43.62393256273761)",246.32
1,1,Voie Verte du canal de Saint-Martory,https://www.af3v.org/les-voies-vertes/voies/12...,CUGNAUX,TOURNEFEUILLE,"2,1km",Voie verte lisse,voie verte,Enrobé,https://sig.af3v.org/index.php/view/embed/?rep...,"('1.32295597087284', '43.5323431084653')","('1.3433622170553097', '43.55502022774329')","(1.333159093964075, 43.5436816681043)",242.65
2,2,Voie Verte de la Masse Pont-du-Casse-Agen,https://www.af3v.org/les-voies-vertes/voies/11...,PONT DU CASSE,AGEN,"2,3km",Voie verte lisse,Voie Verte,Enrobé,https://sig.af3v.org/index.php/view/embed/?rep...,"('0.6480315531906747', '44.216304129821')","('0.6696608866837562', '44.22602284001947')","(0.6588462199372155, 44.22116348492023)",273.37
3,3,Voie Verte des trois ponts Castelnau d’Estréte...,https://www.af3v.org/les-voies-vertes/voies/12...,CASTELNAU D ESTRETEFONDS,FRONTON,"3,2km",Voie verte lisse,Voie Verte,Enrobé,https://sig.af3v.org/index.php/view/embed/?rep...,"('1.35618301280646', '43.76616910015008')","('1.3659247959472003', '43.78084206988599')","(1.3610539043768302, 43.773505585018036)",244.18
4,4,Voie Verte des Berges de l'Ill à Mulhouse,https://www.af3v.org/les-voies-vertes/voies/13...,MULHOUSE,SAUSHEIM,"3,3km",Voie verte lisse,Canal,Enrobé lisse,https://sig.af3v.org/index.php/view/embed/?rep...,"('7.34639', '47.773977')","('7.362202', '47.79706')","(7.354296, 47.7855185)",249.67


In [112]:
cols = ['Nom', 'Lien', 'Distance', 'Type voie', 'Énergie produite annuelle (kWh / m2 de panneau)']
df_simu = df[cols]

In [113]:
df_simu.head()

,Nom,Lien,Distance,Type voie,Énergie produite annuelle (kWh / m2 de panneau)
0,Voie Verte du Chemin d’Encochebérot,https://www.af3v.org/les-voies-vertes/voies/12...,"1,6km",Voie verte lisse,246.32
1,Voie Verte du canal de Saint-Martory,https://www.af3v.org/les-voies-vertes/voies/12...,"2,1km",Voie verte lisse,242.65
2,Voie Verte de la Masse Pont-du-Casse-Agen,https://www.af3v.org/les-voies-vertes/voies/11...,"2,3km",Voie verte lisse,273.37
3,Voie Verte des trois ponts Castelnau d’Estréte...,https://www.af3v.org/les-voies-vertes/voies/12...,"3,2km",Voie verte lisse,244.18
4,Voie Verte des Berges de l'Ill à Mulhouse,https://www.af3v.org/les-voies-vertes/voies/13...,"3,3km",Voie verte lisse,249.67


In [114]:
df_simu['Distance'] = df_simu['Distance'].str.replace('km', '', regex=False).str.replace(',', '.').astype(float)


C:\Users\Nitropc\AppData\Local\Temp\ipykernel_68016\2603737709.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_simu['Distance'] = df_simu['Distance'].str.replace('km', '', regex=False).str.replace(',', '.').astype(float)


In [115]:
df_simu.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 321 entries, 0 to 320
Data columns (total 5 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   Nom                                              321 non-null    object 
 1   Lien                                             321 non-null    object 
 2   Distance                                         321 non-null    float64
 3   Type voie                                        321 non-null    object 
 4   Énergie produite annuelle (kWh / m2 de panneau)  321 non-null    float64
dtypes: float64(2), object(3)
memory usage: 12.7+ KB


In [116]:
df_simu['10% Couverture (Km)'] = round((df_simu['Distance'] * 0.1),2)
df_simu['25% Couverture (Km)'] = round((df_simu['Distance'] * 0.25),2)
df_simu['50% Couverture (Km)'] = round((df_simu['Distance'] * 0.50),2)

C:\Users\Nitropc\AppData\Local\Temp\ipykernel_68016\1788338031.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_simu['10% Couverture (Km)'] = round((df_simu['Distance'] * 0.1),2)
C:\Users\Nitropc\AppData\Local\Temp\ipykernel_68016\1788338031.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_simu['25% Couverture (Km)'] = round((df_simu['Distance'] * 0.25),2)
C:\Users\Nitropc\AppData\Local\Temp\ipykernel_68016\1788338031.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

In [117]:
df_simu.head()

,Nom,Lien,Distance,Type voie,Énergie produite annuelle (kWh / m2 de panneau),10% Couverture (Km),25% Couverture (Km),50% Couverture (Km)
0,Voie Verte du Chemin d’Encochebérot,https://www.af3v.org/les-voies-vertes/voies/12...,1.6,Voie verte lisse,246.32,0.16,0.40,0.80
1,Voie Verte du canal de Saint-Martory,https://www.af3v.org/les-voies-vertes/voies/12...,2.1,Voie verte lisse,242.65,0.21,0.52,1.05
2,Voie Verte de la Masse Pont-du-Casse-Agen,https://www.af3v.org/les-voies-vertes/voies/11...,2.3,Voie verte lisse,273.37,0.23,0.57,1.15
3,Voie Verte des trois ponts Castelnau d’Estréte...,https://www.af3v.org/les-voies-vertes/voies/12...,3.2,Voie verte lisse,244.18,0.32,0.80,1.60
4,Voie Verte des Berges de l'Ill à Mulhouse,https://www.af3v.org/les-voies-vertes/voies/13...,3.3,Voie verte lisse,249.67,0.33,0.82,1.65


In [118]:
df_simu['Production 10% Couverture (kWh)'] = round((( df_simu['10% Couverture (Km)'] * 4.3 ) * df_simu['Énergie produite annuelle (kWh / m2 de panneau)']),2)
df_simu['Production 25% Couverture (kWh)'] = round((( df_simu['25% Couverture (Km)'] * 4.3 ) * df_simu['Énergie produite annuelle (kWh / m2 de panneau)']),2)
df_simu['Production 50% Couverture (kWh)'] = round((( df_simu['50% Couverture (Km)'] * 4.3 ) * df_simu['Énergie produite annuelle (kWh / m2 de panneau)']),2)

C:\Users\Nitropc\AppData\Local\Temp\ipykernel_68016\3753351747.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_simu['Production 10% Couverture (kWh)'] = round((( df_simu['10% Couverture (Km)'] * 4.3 ) * df_simu['Énergie produite annuelle (kWh / m2 de panneau)']),2)
C:\Users\Nitropc\AppData\Local\Temp\ipykernel_68016\3753351747.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_simu['Production 25% Couverture (kWh)'] = round((( df_simu['25% Couverture (Km)'] * 4.3 ) * df_simu['Énergie produite ann

In [119]:
df_simu.rename(columns={
    'Distance' : 'Distance (km)'
},inplace=True)

C:\Users\Nitropc\AppData\Local\Temp\ipykernel_68016\1591108720.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_simu.rename(columns={


In [120]:
df_simu.head()

,Nom,Lien,Distance (km),Type voie,Énergie produite annuelle (kWh / m2 de panneau),10% Couverture (Km),25% Couverture (Km),50% Couverture (Km),Production 10% Couverture (kWh),Production 25% Couverture (kWh),Production 50% Couverture (kWh)
0,Voie Verte du Chemin d’Encochebérot,https://www.af3v.org/les-voies-vertes/voies/12...,1.6,Voie verte lisse,246.32,0.16,0.40,0.80,169.47,423.67,847.34
1,Voie Verte du canal de Saint-Martory,https://www.af3v.org/les-voies-vertes/voies/12...,2.1,Voie verte lisse,242.65,0.21,0.52,1.05,219.11,542.57,1095.56
2,Voie Verte de la Masse Pont-du-Casse-Agen,https://www.af3v.org/les-voies-vertes/voies/11...,2.3,Voie verte lisse,273.37,0.23,0.57,1.15,270.36,670.03,1351.81
3,Voie Verte des trois ponts Castelnau d’Estréte...,https://www.af3v.org/les-voies-vertes/voies/12...,3.2,Voie verte lisse,244.18,0.32,0.80,1.60,335.99,839.98,1679.96
4,Voie Verte des Berges de l'Ill à Mulhouse,https://www.af3v.org/les-voies-vertes/voies/13...,3.3,Voie verte lisse,249.67,0.33,0.82,1.65,354.28,880.34,1771.41


In [121]:
df_simu.to_csv('BDD_PC_Simulation.csv', index=False)